## Packages Required

In [1]:
import numpy as np
import scipy.stats as stats
import sklearn
import matplotlib.pyplot as plt

# SVI Jump-Wings Parameterisation

### Parameters

In [36]:
class SVI():
    def __init__(self, volData, a = 0.1, b = 0.1, rho = 0.1, sig = 0.1, m = 0.1):
        self.volData = volData
        self.a = a
        self.m = m
        self.b = b
        self.rho = rho
        self.sig = sig
        
    def SVI_raw(self, a, b, m, rho, sig):
        totalVol = a + b * (
            rho * (self.volData.strike - m) + np.sqrt((self.volData.strike - m) ** 2) + sig **2)
        impliedVol = totalVol * self.volData.daysExp.astype('int')/250
        return impliedVol
    
    def SVI_JW(self):
        # raw Parameters
        a = self.a
        b = self.b
        rho = self.rho
        m = self.m
        sig = self.sig
        t = self.volData.daysExp.astype('int') / 250
        
        # SVI Jump-Wings
        wt = (self.a + self.b * ( - self.rho * m + np.sqrt(self.m ** 2 + self.sig ** 2)))
        vt = wt / (t)
        phit = (1 / np.sqrt(wt)) * self.b / 2 * ( - self.m / np.sqrt(self.m ** 2 + self.sig ** 2) + self.rho)
        pt = 1 / np.sqrt(wt) * self.b * (1 - self.rho)
        ct = 1 / np.sqrt(wt) * self.b * ( 1+ self.rho)
        vTilde = (self.a + self.b * self.sig * np.sqrt(1 - self.rho ** 2)) / (t)
        
        # Parameterisation
        beta = self.rho - 2 * phit * np.sqrt(wt) / self.b
        alpha = np.sign(beta) * np.sqrt(1/(beta ** 2) - 1)
        
        bNew = np.sqrt(wt) / 2 * (ct + pt)
        rhoNew = 1 - pt * np.sqrt(wt)/b
        mNew = ((vt - vTilde) * t)/(b *(
            - rho + np.sign(alpha) * np.sqrt(1 + alpha ** 2) - alpha * np.sqrt(1 - rho ** 2)))
        aNew = vTilde * t - b * sig * np.sqrt(1 - rho)        
        if m == 0:
            sigNew = (vt * t - a)/b
        else:
            sigNew = alpha * m
            
        impliedVol = self.SVI_raw(a = aNew, b = bNew, m = mNew, rho = rhoNew, sig = sigNew)
        
        return impliedVol
        

        

## Test

In [48]:
priceData = np.load('joinData.pkl', allow_pickle = True)